# BiLSTM with Word2Vec

## Google Drive Access Setup

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!pip3 install gensim
!pip3 install keras
!pip3 install tensorflow

## Word2Vec

In [ ]:
import nltk
nltk.download('punkt')

import pprint
import re
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

from gensim.models import Word2Vec

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Data Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# id = '1ZxiH7Oh6iNm7qu338t9BL045eJ4h0xhR'
# downloaded = drive.CreateFile({'id':id}) 
# downloaded.GetContentFile('Combined_FAANG_percentage_2.2.csv')  
# train_data = pd.read_csv('Combined_FAANG_percentage_2.2.csv', sep=',')

In [ ]:
id = '1r5FJXpiZcbwwjif8fWchrlvFoZzh9ara'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Combined_FAANG_binary_previous.csv')  
train_data = pd.read_csv('Combined_FAANG_binary_previous.csv', sep=',')
print("Number of training examples {}".format(len(train_data)))

Number of training examples 2566858


In [ ]:
train_data.drop(train_data[train_data['Date'] < '2018-07-20'].index, inplace = True)
train_data

,symbol,message,datetime,user,message_id,Date,Time,label
1,AAPL,aapl watching gap fill 169 20,2018-11-24 07:02:32,1665234.0,146068732.0,2018-11-24,07:02:32,1
3,AAPL,aapl,2020-01-27 07:07:03,1229493.0,191978042.0,2020-01-27,07:07:03,0
5,AAPL,aapl loads cash hand great service business lo...,2018-11-01 23:39:14,123291.0,143688765.0,2018-11-01,23:39:14,1
8,AAPL,qq became euphoric calls exp week aiming ath f...,2020-05-13 02:13:00,2250451.0,212222428.0,2020-05-13,02:13:00,0
9,AAPL,spy novices like davey day trader lose money s...,2020-06-24 11:12:09,543250.0,222404886.0,2020-06-24,11:12:09,0
...,...,...,...,...,...,...,...,...
2566843,NFLX,nflx last 3 months big green candles followed ...,2019-05-01 18:21:25,637003.0,162589986.0,2019-05-01,18:21:25,1
2566848,NFLX,nflx full move key support,2020-03-12 16:52:43,677915.0,199933357.0,2020-03-12,16:52:43,0
2566849,NFLX,spy spx nflx nvda virtually volume today absen...,2019-10-14 18:16:28,55818.0,180328889.0,2019-10-14,18:16:28,1
2566853,NFLX,nflx sister owns kinda thinking telling sell b...,2019-01-11 20:51:22,607557.0,150426203.0,2019-01-11,20:51:22,1


In [ ]:
#use this when using percentage label with 2 labels
train_data.drop(train_data[train_data['label'] == 0].index, inplace = True)
train_data["label"].replace({-1: 0}, inplace=True)

In [ ]:
train_data = train_data.sample(frac=1)
train_data

,symbol,message,datetime,user,message_id,Date,Time,label
675348,AAPL,free video setups march 23 quot buy quot spy a...,2020-03-22 23:37:25,50220.0,202007251.0,2020-03-22,23:37:25,0
2219386,NFLX,nflx puts earning call,2019-01-17 20:09:54,1759086.0,150999143.0,2019-01-17,20:09:54,1
1781982,FB,dis visit instagram page see full post full an...,2020-02-02 03:01:23,2263823.0,192944465.0,2020-02-02,03:01:23,1
1690576,FB,dis frozen 2 pop stock coming weekend going pr...,2019-11-19 18:59:16,1070521.0,184280936.0,2019-11-19,18:59:16,1
1939630,NFLX,mostly higher midday mu xlc fb goog vz nflx dis,2018-09-21 16:56:48,8115.0,138290085.0,2018-09-21,16:56:48,0
...,...,...,...,...,...,...,...,...
1959069,NFLX,nflx 350,2020-01-23 20:59:37,3008740.0,191633184.0,2020-01-23,20:59:37,1
1957688,NFLX,nflx insane,2020-01-21 20:55:18,1317171.0,191255702.0,2020-01-21,20:55:18,0
339081,AAPL,aapl goodbye apple development apps apple plat...,2020-03-29 02:56:47,655693.0,203351339.0,2020-03-29,02:56:47,1
1499784,AMZN,amzn aapl nke lol complaining nike managed bri...,2018-09-04 23:45:13,1588778.0,136123796.0,2018-09-04,23:45:13,1


In [ ]:
train_data = train_data.dropna(subset=['message']).reset_index()  
train_data

,index,symbol,message,datetime,user,message_id,Date,Time,label
0,675348,AAPL,free video setups march 23 quot buy quot spy a...,2020-03-22 23:37:25,50220.0,202007251.0,2020-03-22,23:37:25,0
1,2219386,NFLX,nflx puts earning call,2019-01-17 20:09:54,1759086.0,150999143.0,2019-01-17,20:09:54,1
2,1781982,FB,dis visit instagram page see full post full an...,2020-02-02 03:01:23,2263823.0,192944465.0,2020-02-02,03:01:23,1
3,1690576,FB,dis frozen 2 pop stock coming weekend going pr...,2019-11-19 18:59:16,1070521.0,184280936.0,2019-11-19,18:59:16,1
4,1939630,NFLX,mostly higher midday mu xlc fb goog vz nflx dis,2018-09-21 16:56:48,8115.0,138290085.0,2018-09-21,16:56:48,0
...,...,...,...,...,...,...,...,...,...
1023248,1959069,NFLX,nflx 350,2020-01-23 20:59:37,3008740.0,191633184.0,2020-01-23,20:59:37,1
1023249,1957688,NFLX,nflx insane,2020-01-21 20:55:18,1317171.0,191255702.0,2020-01-21,20:55:18,0
1023250,339081,AAPL,aapl goodbye apple development apps apple plat...,2020-03-29 02:56:47,655693.0,203351339.0,2020-03-29,02:56:47,1
1023251,1499784,AMZN,amzn aapl nke lol complaining nike managed bri...,2018-09-04 23:45:13,1588778.0,136123796.0,2018-09-04,23:45:13,1


In [ ]:
#Tokenization of sentences
target_text = train_data['message'].values

# Tokenising each sentence
sentences=[]
sentences=[word_tokenize(sentence) for sentence in target_text]

# Print 10 tokenised sentences
print(sentences[:10])

[['free', 'video', 'setups', 'march', '23', 'quot', 'buy', 'quot', 'spy', 'aapl', 'fb', 'msft'], ['nflx', 'puts', 'earning', 'call'], ['dis', 'visit', 'instagram', 'page', 'see', 'full', 'post', 'full', 'analysis', 'reports', 'well', 'including', 'swot', 'analysis', 'revenue', 'history', 'bulls', 'vs', 'bears', 'sentiment', 'replacement', 'head', 'start', 'due', 'diligence'], ['dis', 'frozen', '2', 'pop', 'stock', 'coming', 'weekend', 'going', 'profitable', 'load', 'calls', 'run', 'thinking', '155', 'friday'], ['mostly', 'higher', 'midday', 'mu', 'xlc', 'fb', 'goog', 'vz', 'nflx', 'dis'], ['aapl', 'said', '39', 'say', 'sqq', 'inverse', 'aapl', 'stock', 'go', 'get', 'money'], ['aapl', 'joke', 'wait', 'lower', 'guidance', 'face', 'tears', 'joy'], ['week', '39', 'top', 'stock', 'picks', 'netflix', 'marten', 'transport', 'nflx', 'mrtn'], ['amzn', 'holy', 'crap', '39', 'expect', 'much', 'today', 'gone', 'since', '1pm', '39', 'think', 'would', 'see', 'much', '3150', 'lol'], ['tsla', 'aapl', 

### Word2Vec - Skip Gram

In [ ]:
wv_sg_model = Word2Vec(sentences=sentences, size=100, window=5, min_count=5, workers=4, sg=1)

In [ ]:
similar_words=wv_sg_model.wv.most_similar("aapl")
pprint.pprint(similar_words)

[('appl', 0.7539772987365723),
 ('nflx', 0.7319340705871582),
 ('tks', 0.7004857063293457),
 ('nxd', 0.6979733109474182),
 ('fb', 0.69671231508255),
 ('spy', 0.694193959236145),
 ('uyg', 0.6859781742095947),
 ('djx', 0.6855261325836182),
 ('merrier', 0.6847277879714966),
 ('naaging', 0.684084415435791)]


In [ ]:
#Save Word2Vec model
wv_sg_model.save('wvsgmodel1y')
wv_sg_model.wv.save_word2vec_format('wvsgmodel1y'+".bin",binary=True)

In [ ]:
#Reload Word2Vec model
import gensim
from gensim.models.keyedvectors import KeyedVectors
wv_sg_model = Word2Vec.load('wvsgmodel1y')
wv_sg_model = KeyedVectors.load_word2vec_format('wvsgmodel1y'+".bin", binary=True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data['message'].values, train_data['label'].values, test_size=0.1, random_state=42)

In [ ]:
df_train = pd.DataFrame(y_train,columns=['label'])
df_train['label'].value_counts()

1    494380
0    426547
Name: label, dtype: int64

In [ ]:
df_test = pd.DataFrame(y_test,columns=['label'])
df_test['label'].value_counts()

1    54997
0    47329
Name: label, dtype: int64

### BiLSTM

In [ ]:
import os
import re
import csv
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
import tensorflow as tf

In [ ]:
MAX_SEQUENCE_LENGTH = 160
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.1

In [ ]:
from keras.utils.np_utils import to_categorical
print('Found %s word vectors of word2vec' % len(wv_sg_model.wv.vocab))
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(sentences)

sequences_1 = tokenizer.texts_to_sequences(X_train)

sequences_test = tokenizer.texts_to_sequences(X_test)


word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))
data_1 = pad_sequences(sequences_1, maxlen=MAX_SEQUENCE_LENGTH)

data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y_train))

labels_test = to_categorical(np.asarray(y_test))
print('Shape of data tensor:', data_1.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data_1.shape[0])
np.random.shuffle(indices)
data = data_1[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

Found 34422 word vectors of word2vec
Found 104508 unique tokens
Shape of data tensor: (920927, 160)
Shape of label tensor: (920927, 2)


In [ ]:
print('Traing and validation set number of positive and negative reviews')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Traing and validation set number of positive and negative reviews
[383715. 445120.]
[42832. 49260.]


In [ ]:
print('Preparing embedding matrix')

nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in wv_sg_model.wv.vocab:
        embedding_matrix[i] = wv_sg_model.wv.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Preparing embedding matrix
Null word embeddings: 70142


In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(2, activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
print("model fitting - Bidirectional LSTM")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=2, batch_size=32)

model fitting - Bidirectional LSTM
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 160, 100)          10450900  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 2)                 402       
Total params: 10,612,102
Trainable params: 10,612,102
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
25902/25902 [==============================] - 7572s 292ms/step - loss: 0.6831 - acc: 0.5592 - val_loss: 0.6797 - val_acc: 0.5656
Epoch 2/2
25902/25902 [==============================] - 7499s 29

### Attention layer

In [ ]:
# Attention network		  
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==3
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros") 
        super(AttLayer, self).build(input_shape) 

    def call(self, x, mask=None):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_gru = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer()(l_gru)
preds = Dense(2, activation='softmax')(l_att)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention network")
model.summary()
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=32)

model fitting - attention GRU network
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 160, 100)          10450900  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 160, 200)          121200    
_________________________________________________________________
att_layer (AttLayer)         (None, 200)               360       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 10,572,862
Trainable params: 10,572,862
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
25902/25902 [==============================] - 7281s 281ms/s

In [ ]:
########################################
## Classification report
########################################
from sklearn.metrics import classification_report
print('Start making the submission before fine-tuning')

preds = model.predict(data_test, batch_size=32, verbose=1)

pred_flat = np.argmax(preds, axis=1).flatten()

print(pred_flat)
print(classification_report(y_test, pred_flat))

Start making the submission before fine-tuning
3198/3198 [==============================] - 171s 53ms/step
[1 0 1 ... 1 1 1]
              precision    recall  f1-score   support

           0       0.58      0.26      0.36     47329
           1       0.57      0.83      0.68     54997

    accuracy                           0.57    102326
   macro avg       0.57      0.55      0.52    102326
weighted avg       0.57      0.57      0.53    102326

